In [2]:
import sys, os, time
#%load_ext autoreload
#%autoreload 2
import foundation as fd
import foundation.util as util
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import torchvision
import h5py as hf
from itertools import product

import ZODB, ZODB.FileStorage
import ZEO
import neo
import transaction
import persistent
from persistent.list import PersistentList
from BTrees import OIBTree, IOBTree, IIBTree

In [222]:
s = IOBTree.Set()

In [223]:
s.add(323)

1

In [227]:
list(s.keys())

[3, 4, 323]

In [226]:
s.update([3,4])

2

In [207]:
def populate(tree, N=10000000):
    for i in range(N):
        tree[i] = i+1
    return tree
    

In [208]:
d = PersistentMapping()

In [209]:
len(d), list(d.items())

(0, [])

In [210]:
s1 = IIBTree.BTree()
s2 = PersistentMapping()

In [211]:
list(s1.items()), list(s2.items())

([], [])

In [212]:
#t1, t2 = 0, 0

start = time.time()
populate(s1)
print('s1: {}'.format(time.time() - start))

start = time.time()
populate(s2)
print('s2: {}'.format(time.time() - start))

s1: 1.476391077041626
s2: 6.2712836265563965


In [12]:
t = IIBTree.BTree()

In [13]:
list(t.items())

[]

In [15]:
t[3] = 546
t[4] = 657

In [16]:
list(t.items())

[(3, 546), (4, 657)]

In [17]:
len(t)

2

In [5]:
# top level objects:
# - ID table (name -> ID)
# - ID list (ID -> name)
# - rID table (redir -> rID)
# - rID list (rID -> redir)
# - num list (ID -> num)
# - redirect list (ID -> redirects)
# - link list (ID -> links)

In [4]:
addr, stop = ZEO.server(path='test.zeo')
addr, stop

(('127.0.0.1', 37379),
 <function ZEO._forker.start_zeo_server.<locals>.stop(stop_timeout=99)>)

In [167]:
db = ZEO.DB(addr)
connection = db.open()
root = connection.root

In [168]:
root.test = OIBTree.BTree()
root.test2 = OIBTree.BTree()
transaction.commit()

In [170]:
root.test, root.test2

(<BTrees.OIBTree.OIBTree object at 0x7f5e786efe18 oid 0xd1 in <Connection at 7f5e786ee668>>,
 <BTrees.OIBTree.OIBTree object at 0x7f5e786efae8 oid 0xd0 in <Connection at 7f5e786ee668>>)

In [71]:
#transaction.abort()

In [127]:
def g(n,x, addr):
    print(n, x, addr)
    return 3

In [182]:
def f(ID, args):
    x, n, addr = args
    try:
        db = ZEO.DB(addr)
        #conn = db.open()
        
#         for attempt in transaction.manager.attempts():
#             with attempt:
        
        
    
        with db.transaction() as conn:
            
            if x:
                d, l = conn.root.test, 'test'
            else:
                d, l = conn.root.test2, 'test2'

            print('{} before: {}'.format(ID, l), d.get(n, '** no {}'.format(n)))
            
            if n not in d:
                d[n] = -1
            
            d[n] += 1

            time.sleep(0.02)
            print('{} after: {}'.format(ID, l), d.get(n, '** no {}'.format(n)))


            #transaction.commit()
        
        print('done')
        
    except Exception as e:
        return e

In [183]:
pool = mp.Pool(9)

0 before: test2 0
5 before: test 1
2 before: test2 0
1 before: test2 0
4 before: test 0
3 before: test 1
6 before: test 1
8 before: test 1
7 before: test 0
5 after: test 2
0 after: test2 1
2 after: test2 1
1 after: test2 1
3 after: test 2
4 after: test 1
6 after: test 2
8 after: test 2
7 after: test 1
done
done
done
done
done
done
0 before: test2 1
3 before: test 2
2 before: test2 1
4 before: test 1
1 before: test2 1
5 before: test 2
0 after: test2 2
3 after: test 3
2 after: test2 2
4 after: test 2
1 after: test2 2
5 after: test 3
done
done
done
done
done
done


In [187]:
N = 5
jobs = enumerate(product(range(2), ['a', 'b', 'c'], [addr]))
jobs

In [188]:
#f(*jobs[0])

In [189]:
pool.starmap(f, jobs)

[None, None, None, None, None, None]

In [58]:
root.a, root.b, root.c

('0', '1', '1')

In [57]:
root

<root: ID_table a article_list b c link_list num_list rID_list ...>

In [171]:
db.close()
#stop()

In [22]:
root

<root: >

In [41]:
root.ID_table = PersistentMapping()
root.rID_table = PersistentMapping()
root.link_list = PersistentList()
root.redir_list = PersistentList()
root.num_list = PersistentList()
root.article_list = PersistentList()
root.rID_list = PersistentList()

In [42]:
for i in range(10000):
    root.rID_list.append(i)

In [53]:
len(root.rID_list)

10000

In [44]:
transaction.commit()

In [20]:
db.close()

In [ ]:
class WikiGraph_DB(object):
    def __init__(self, db_path=None, db=None):
        
        if db is None:
            db  = ZODB.DB(db_path)
        self.db = db
        self.conn = self.db.open()
        self.root = self.conn.root
        
    def add_node(*names):
        pass
    
    def add_redir(*redir):
        pass
        
    def add(name, links, redirs):
        
        ID = self.add_node(name)
        
        links = self.add_node(links)
        
        transaction.commit()
        
    
    def increment(*IDs):
        pass
    
    def get_ID(*names):
        pass
    
    def get_name(*IDs):
        pass
    
    def get_num(*IDs):
        pass

In [2]:
class Article(persistent.Persistent):
    def __init__(self, name, links=None, redirs=None):
        self.name = name
        self.num = 1
        self.links = links
        self.redirs = redirs
        
    def inc(self):
        self.num += 1
        
class Article_Graph(persistent.Persistent):
    
    def __init__(self):
        self.articles = [] # ID -> article
        self.IDs = OIBTree.BTree() # name -> ID
        self.next_ID = 0
    
    def items(self):
        return iter(self.articles)
    
    def __getitem__(self, x):
        return self.articles[x] # TODO: should fail if x is a string, then use try except
    
    def __len__(self):
        return len(self.articles)
    
    def add_article(self, name, links, redirs):
        ID = self.IDs.get(name, None)
        
        if ID is None:
            self.IDs[article.name] = self.next_ID
            self.next_ID += 1
            
            links = self.get_ID(links)
            redirs = self.get_ID()
            
            article = Article(name, )
            
            self.articles.append()
    
    def add_articles(self, articles):
        
        for article in articles:
            ID = self.IDs.get(article.name, None)
            
            if ID is None:
                self.IDs[article.name] = self.next_ID
                self.next_ID += 1
                
                
                
    
    def get_ID(self, *names):
        
        for name in names:
            ID = self.IDs.get(name, None)
            
    
    def items(self):
        self.cursor.execute('''SELECT * FROM articles''')
        return iter(self.cursor)
        
    def add(self, *names):
        if self.lock is not None:
            self.lock.acquire()
        for name in names:
            try:
                #print(_insert_temp, name)
                self.cursor.execute(_insert_temp, (name,))
            except sqlite3.IntegrityError:
                pass
        #self.cursor.executemany(_insert_temp, [(n,) for n in names])
        self.conn.commit()
        if self.lock is not None:
            self.lock.release()
        
    def increment(self, *names):
        if self.lock is not None:
            self.lock.acquire()
        self.cursor.execute(_inc_temp.format(', '.join('?'*len(names))), names)
        self.conn.commit()
        if self.lock is not None:
            self.lock.release()
        
    def __len__(self):
        self.cursor.execute('''SELECT COUNT(*) FROM articles''')
        return self.cursor.fetchall()[0][0]
        
    def get_id(self, *names):
        self.cursor.execute(_sel_id_temp.format(', '.join('?'*len(names))), names)
        return self.cursor.fetchall()
        
    def get_name(self, *IDs):
        self.cursor.execute(_sel_name_temp.format(', '.join('?'*len(IDs))), IDs)
        return self.cursor.fetchall()
    
    def get_full_from_names(self, *names):
        self.cursor.execute(_sel_full_name_temp.format(', '.join('?'*len(names))), names)
        return self.cursor.fetchall()
    
    def get_full_from_ids(self, *IDs):
        self.cursor.execute(_sel_full_id_temp.format(', '.join('?'*len(IDs))), IDs)
        return self.cursor.fetchall()
    
    def __del__(self):
        self.conn.close()

In [3]:
# storage = ZODB.FileStorage.FileStorage('mydata.fs')
# db = ZODB.DB(storage)
# connection = db.open()
# root = connection.root